In [1]:
from device import Device
from rgf import GreensFunction
import scipy as sp
from helper import Helper_functions
import scipy.sparse as spa
import numpy as np
import scipy.sparse as sp

In [2]:
device = Device()
rgf = GreensFunction(device_state=device)

In [3]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 15164 stored elements and shape (560, 560)>
  Coords	Values
  (0, 0)	(7.8483199999999975+0j)
  (0, 1)	(-6.661338147750939e-16+0j)
  (0, 2)	(1.1102230246251565e-15+0j)
  (0, 3)	(-9.999999999999996+0j)
  (0, 10)	(-1.9352073963782712-0.30650674038727554j)
  (0, 11)	(1.7253359913073014+0.27326637535498594j)
  (0, 12)	(1.7253359913073014+0.27326637535498594j)
  (0, 13)	(1.7253359913073014+0.27326637535498594j)
  (0, 14)	(-1.3029177291723641-0.20636189532388063j)
  (0, 15)	(-1.3029177291723641-0.20636189532388063j)
  (0, 16)	(-1.3029177291723641-0.20636189532388063j)
  (0, 18)	(-2.5054621766260616e-16-3.9682622459921325e-17j)
  (0, 19)	(-1.503557960887978-0.23814018613074345j)
  (1, 0)	(-6.661338147750939e-16+0j)
  (1, 1)	(14.229249999999999+0j)
  (1, 2)	(-9.999999999999998+0j)
  (1, 3)	(-8.881784197001252e-16+0j)
  (1, 10)	(-1.7253359913073014-0.27326637535498594j)
  (1, 11)	(0.3514919420620609+0.055670854522083854j)
  (1, 12)

In [4]:
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)

[[ 7.84832000e+00+0.j -6.66133815e-16+0.j  1.11022302e-15+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 8.88178420e-16+0.j  1.42292500e+01+0.j -1.00000000e+01+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.00000000e+01+0.j  1.42292500e+01+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 ...
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   1.37895000e+01+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  1.37895000e+01+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  1.91165000e+01+0.j]]


In [ ]:
def surface_gf(Energy, H00, H10, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """

    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0
    first_time = True

    while err > tol:
        if first_time:
            inv_E = Helper_functions.sparse_inverse(spa.csr_matrix(Energy * I) - spa.csr_matrix(epsilon))
            first_time = False
        else:

            inv_E = np.linalg.solve(Energy * I - epsilon, I)
    
        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)


surface_gf(0, H00, H10, tol=1e-3)

start
[[-0.46265481-3.43058915e-15j  0.26020359+1.07328341e-01j
   0.22438539+1.30259377e-01j ... -0.06810719-6.98355909e-03j
   0.0692144 -3.15319604e-02j -0.14372304-2.32399663e-02j]
 [ 0.26020359-1.07328341e-01j -0.23397533-5.30563211e-16j
  -0.18724291-1.82886728e-02j ...  0.02738157-1.92212765e-02j
  -0.01557864+3.36768101e-02j  0.06061459-3.72506451e-02j]
 [ 0.22438539-1.30259377e-01j -0.18724291+1.82886728e-02j
  -0.23001246-7.12869148e-16j ...  0.02137388-2.46542704e-02j
  -0.00679168+3.54891358e-02j  0.04851731-4.95490009e-02j]
 ...
 [-0.06810719+6.98355909e-03j  0.02738157+1.92212765e-02j
   0.02137388+2.46542704e-02j ... -0.09098765-1.23251985e-16j
   0.01310373-6.53896350e-03j -0.03164499-2.48658897e-03j]
 [ 0.0692144 +3.15319604e-02j -0.01557864-3.36768101e-02j
  -0.00679168-3.54891358e-02j ...  0.01310373+6.53896350e-03j
  -0.09913441-1.44218598e-16j  0.02828178+1.64868977e-02j]
 [-0.14372304+2.32399663e-02j  0.06061459+3.72506451e-02j
   0.04851731+4.95490009e-02j ... -0

array([[-0.19135261+7.48661586e-02j,  0.2118514 -9.00054160e-02j,
         0.21315018-8.91832769e-02j, ..., -0.00394714-6.10309501e-03j,
         0.00682916+6.33059985e-03j, -0.00664039-1.52196826e-02j],
       [ 0.08016425+1.05039217e-01j, -0.08153963-1.07665448e-01j,
        -0.03719187-1.07388794e-01j, ...,  0.00311232+1.45570970e-03j,
        -0.00432909-8.31659925e-06j,  0.00699558+3.99447844e-03j],
       [ 0.04821244+1.38463126e-01j, -0.0066478 -1.39331436e-01j,
        -0.04586694-1.39069810e-01j, ...,  0.00346526+5.96738055e-04j,
        -0.00390684+1.34903520e-03j,  0.00737847+2.26683188e-03j],
       ...,
       [ 0.02583285+1.58618095e-02j, -0.03115715-1.38741753e-02j,
        -0.03105831-1.48418945e-02j, ..., -0.07339599+1.93945977e-03j,
         0.00216878+5.16391693e-03j, -0.00123908-1.48397039e-03j],
       [ 0.03396117-3.76533231e-02j, -0.04233916+3.50477663e-02j,
        -0.04305065+3.51312218e-02j, ..., -0.00134742+1.44865477e-03j,
        -0.08046237-2.50662427e-03j

In [7]:
def surface_gf(Energy, H00 : np.ndarray, H10: np.ndarray, tol=1e-6): 
    """ 
    This iteratively calculates the surface green's function for the lead based. 
    Although it is tested for 1D, it should be good for 2D surfaces. 
    """
    
    Energy = Energy
    dagger = lambda A: np.conjugate(A.T)
    
    I = np.eye(H00.shape[0], dtype=complex)
    H01 = dagger(H10)

    epsilon_s = H00.copy()
    epsilon = H00.copy()
    alpha = H01.copy()
    beta = dagger(H10).copy()
    err = 1.0

    while err > tol:
        inv_E = np.linalg.solve(Energy * I - epsilon, I)

        epsilon_s_new = epsilon_s + alpha @ inv_E @ beta
        epsilon_new = epsilon + beta @ inv_E @ alpha + alpha @ inv_E @ beta
        alpha_new = alpha @ inv_E @ alpha
        beta_new = beta @ inv_E @ beta

        err = np.linalg.norm(alpha_new, ord='fro')

        epsilon_s, epsilon, alpha, beta = epsilon_s_new, epsilon_new, alpha_new, beta_new

    return  np.linalg.solve(Energy * I - epsilon_s, I)

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)

surface_gf(0, H00, H10)

array([[-0.19135261+7.48661586e-02j,  0.2118514 -9.00054160e-02j,
         0.21315018-8.91832769e-02j, ..., -0.00394714-6.10309501e-03j,
         0.00682916+6.33059985e-03j, -0.00664039-1.52196826e-02j],
       [ 0.08016425+1.05039217e-01j, -0.08153963-1.07665448e-01j,
        -0.03719187-1.07388794e-01j, ...,  0.00311232+1.45570970e-03j,
        -0.00432909-8.31659933e-06j,  0.00699558+3.99447844e-03j],
       [ 0.04821244+1.38463126e-01j, -0.0066478 -1.39331436e-01j,
        -0.04586694-1.39069810e-01j, ...,  0.00346526+5.96738055e-04j,
        -0.00390684+1.34903520e-03j,  0.00737847+2.26683188e-03j],
       ...,
       [ 0.02583285+1.58618095e-02j, -0.03115715-1.38741753e-02j,
        -0.03105831-1.48418945e-02j, ..., -0.07339599+1.93945977e-03j,
         0.00216878+5.16391693e-03j, -0.00123908-1.48397039e-03j],
       [ 0.03396117-3.76533231e-02j, -0.04233916+3.50477663e-02j,
        -0.04305065+3.51312218e-02j, ..., -0.00134742+1.44865477e-03j,
        -0.08046237-2.50662427e-03j

In [8]:
from helper import Helper_functions

H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=True)
print(H00)
Helper_functions.sparse_inverse(-H00)
H00,H10 = device.hamiltonian.get_H00_H01(ky=0.1, sparse=False)
print(H00)
Helper_functions.sparse_inverse(-H00)

<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 15164 stored elements and shape (560, 560)>
  Coords	Values
  (0, 0)	(7.8483199999999975+0j)
  (0, 1)	(-6.661338147750939e-16+0j)
  (0, 2)	(1.1102230246251565e-15+0j)
  (0, 3)	(-9.999999999999996+0j)
  (0, 10)	(-1.9352073963782712-0.30650674038727554j)
  (0, 11)	(1.7253359913073014+0.27326637535498594j)
  (0, 12)	(1.7253359913073014+0.27326637535498594j)
  (0, 13)	(1.7253359913073014+0.27326637535498594j)
  (0, 14)	(-1.3029177291723641-0.20636189532388063j)
  (0, 15)	(-1.3029177291723641-0.20636189532388063j)
  (0, 16)	(-1.3029177291723641-0.20636189532388063j)
  (0, 18)	(-2.5054621766260616e-16-3.9682622459921325e-17j)
  (0, 19)	(-1.503557960887978-0.23814018613074345j)
  (1, 0)	(-6.661338147750939e-16+0j)
  (1, 1)	(14.229249999999999+0j)
  (1, 2)	(-9.999999999999998+0j)
  (1, 3)	(-8.881784197001252e-16+0j)
  (1, 10)	(-1.7253359913073014-0.27326637535498594j)
  (1, 11)	(0.3514919420620609+0.055670854522083854j)
  (1, 12)

array([[-0.0090192 -5.10008702e-17j,  0.10186053+2.42719853e-03j,
         0.09749155+2.67674066e-03j, ...,  0.0014451 -4.54025400e-04j,
        -0.00222137-1.92015229e-03j,  0.00086335+2.20492192e-03j],
       [ 0.10186053-2.42719853e-03j, -0.10101744+3.55248541e-18j,
        -0.06147344+9.38219590e-05j, ...,  0.00029316-1.49862273e-04j,
        -0.00054636-3.29359108e-04j,  0.00026605+4.40402709e-04j],
       [ 0.09749155-2.67674066e-03j, -0.06147344-9.38219590e-05j,
        -0.10234001+4.73555206e-18j, ...,  0.00018871-1.26928282e-04j,
        -0.00040153-1.80788273e-04j,  0.00021862+2.80061504e-04j],
       ...,
       [ 0.0014451 +4.54025400e-04j,  0.00029316+1.49862273e-04j,
         0.00018871+1.26928282e-04j, ..., -0.06489748-4.13012478e-18j,
        -0.00102763-3.67162632e-04j,  0.0004483 -9.76825058e-05j],
       [-0.00222137+1.92015229e-03j, -0.00054636+3.29359108e-04j,
        -0.00040153+1.80788273e-04j, ..., -0.00102763+3.67162632e-04j,
        -0.07130404-8.87536834e-19j

In [4]:
H = device.hamiltonian.create_sparse_hamlitonian(0.1)